# Assignment 3 : Linear Optimization

Name : Ahmik Virani  <br>
Roll Number : ES22BTECH11001

In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import null_space

In [ ]:
# Here I define a function to convert the vertex in the new space back to the original space
def convert_vertex(vertex, n):
  new_vertex = np.zeros(n//2)
  for i in range(n):
    if(i%2 == 0):
      new_vertex[i//2] += vertex[i]
    else:
      new_vertex[i//2] -= vertex[i]
  return new_vertex

def handle_degeneracy(epsilon, B_original, m):
  B_new = np.zeros(m)
  for i in range(m):
    B_new[i] = B_original[i] + pow(epsilon, i+1)
  return B_new

# This is a function which takes the initial feasable point to the vertex
def go_to_vertex(z, A, B, m, n):

  path = []
  current_z = z.copy()

  while True:
    # We first need to find the set of tight and untight rows of A
    # Let A1 be the set of tight rows
    # Let A2 be the set of untight rows
    Az = A @ current_z
    A1_indexes = []
    A2_indexes = []
    for i in range(m):
        # For floats, we need to check if they are very close
        if abs(Az[i] - B[i]) < 1e-9:
            A1_indexes.append(i)
        else:
            A2_indexes.append(i)

    A1 = A[A1_indexes, :]
    A2 = A[A2_indexes, :]

    # Ensure that the number of tight rows is not zero
    if len(A1_indexes) == 0:
      rank = 0
    else:
      rank = np.linalg.matrix_rank(A1)

    # If rank of the tight rows is >= n, then this is already a vertex
    if rank >= n:
      if not np.allclose(current_z, z, atol=1e-9):
        path.append(np.round(current_z, 9))
      return (current_z, path)

    # We need to find a direction in the null space of A1
    if rank == 0:
      # We need to move to a random direction
      d = np.random.rand(n)
      d = d / np.linalg.norm(d)
    else:
      d = null_space(A1)[:, 0]

    min_step_size = np.inf

    # If we go towards d or opposite d
    direction = 0

    # First check going towards d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = 1

    # Next check going opposite to d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], -d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = -1

    current_z += min_step_size * direction * d

# The extra parameter original is just to ensure that we do not need to convert the vertex back to original space
# This is just needed to differentiate if we need to find vertex or actually solve the simplex algo after someone given us z
def simplex(A, B, c, z, original):
  m, n = A.shape

  # First let me introduce a variable epsilon
  epsilon = 1e-1

  # Let me also keep track of the original B
  B_original = B.copy()

  # Since we are handling degeneracy, we need an additional outer loop where we restart if we have picked the wrong epsilom
  while(True):
    # We are already given the starting vertex
    current_vertex = z.copy()

    # Ensure to update B every time the outer loop runs
    B = handle_degeneracy(epsilon, B_original, m)

    current_vertex, finding_vertex_path = go_to_vertex(current_vertex, A, B, m, n)

    print(f"Applying Simplex with epsilon = {epsilon}")

    # A variable done to break out of this outer loop
    # If done = 1, we have a finite optimum
    # If done = 2, we have unbounded/infinite optimum
    done = 0

    # Variable to keep track of visited vertices and its corresponding value of objective function
    visited_vertices = []
    visited_values = []

    while(True):

      # We first need to find the set of tight and untight rows of A
      # Let A1 be the set of tight rows
      # Let A2 be the set of untight rows
      Az = A @ current_vertex
      A1_indexes = []
      A2_indexes = []
      for i in range(m):
          # For floats, we need to check if they are very close
          if abs(Az[i] - B[i]) < 1e-9:
              A1_indexes.append(i)
          else:
              A2_indexes.append(i)

      A1 = A[A1_indexes, :]
      A2 = A[A2_indexes, :]

      # Check if the current vertex if degenrate
      # Which means that more than n constraints are tight
      if len(A1_indexes) != n:
        for i in range(len(finding_vertex_path)):
          if original == True:
            print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i], n), 9))
          else:
            print("Trying to reach a vertex before applying Simplex - Modified Problem. Moving to: ", np.round(finding_vertex_path[i], 9))
        for i in range(len(visited_vertices)):
          print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
        # We need to update our epsilon
        epsilon /= 2
        print(f"Degeneracy found. Reducing episilon to : {epsilon}")
        print("=============================================================")
        print("\n\n")
        # Then we restart from the outer loop
        break

      # We know that rank of A1 is n
      # Thus, we know that A1 has as inverse
      A_inv = np.linalg.inv(A1)

      x_original = np.linalg.solve(A1, B_original[A1_indexes])
      if original:
        visited_vertices.append(convert_vertex(x_original, n))
        visited_values.append(np.dot(c, x_original))
      else:
        visited_vertices.append(x_original)
        visited_values.append(np.dot(c, x_original))

      # Next we check c.T dot A_inv
      # If even one is negative, then z is not optimum
      all_positive = True

      for row in A_inv.T:
        if np.dot(c.T, row) < 0:
          all_positive = False
          # Move in opposite direction of this
          # Let d be direction of movement
          d = -row

          # We also need the size of movement
          # But we only check along the untight rows, i.e. A2
          step_sizes = []
          for i in A2_indexes:
            Ai_dot_d = np.dot(A[i], d)
            # We move in this direction if it becomes more tight
            if Ai_dot_d > 1e-9:
              step_sizes.append((B[i] - Az[i]) / Ai_dot_d)

          if step_sizes:
              # We choose the minimum step size to ensure we are going to the adjacent vertex
              alpha = min(step_sizes)
              current_vertex += alpha * d
              break
          else:
            # Here, we have not found any direction where rows of A2 become more tight
            # Thus this optimum is unbounded and we report this
            done = 2
            break

      # Finally we even need to check in the original system if the rows of A1 are tight
      if all_positive is True:
          done = 1

      if done > 0:
        if done == 1:
          for i in range(len(finding_vertex_path)):
            if original == True:
              print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i], n), 9))
            else:
              print("Trying to reach a vertex before applying Simplex - Modified Problem. Moving to: ", np.round(finding_vertex_path[i], 9))
          for i in range(len(visited_vertices)):
            print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
          return (visited_vertices[-1], visited_values[-1])
        else:
          for i in range(len(finding_vertex_path)):
            if original == True:
              print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i], n), 9))
            else:
              print("Trying to reach a vertex before applying Simplex - Modified Problem. Moving to: ", np.round(finding_vertex_path[i], 9))
          for i in range(len(visited_vertices)):
            print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
          print("Optimum is unbounded")
          return (-1, np.inf)
        break

    if done > 0:
      break

In [ ]:
# Read CSV file
df = pd.read_csv('Testcase.csv', header=None)

# Get total rows and columns
rows, cols = df.shape
m = rows - 1
n = cols - 1

# Extract components
c = df.iloc[0, :-1].dropna().to_numpy(dtype=float)
A = df.iloc[1:, :-1].to_numpy(dtype=float)
B = df.iloc[1:, -1].to_numpy(dtype=float)

print("n =", n)
print("m =", m)
print("c =", c)
print("A =\n", A)
print("B =", B)


n = 2
m = 5
c = [3. 7.]
A =
 [[ 1.  0.]
 [ 0.  1.]
 [ 2.  1.]
 [ 1.  2.]
 [-1. -1.]]
B = [ 4.  4.  8.  4. -1.]


First Let us handle the assumption that rank(A) = n <br>
To do this, we have seen in class that we can split each variable into sum of two non-negative variables (eg. Split x = x1 - x2, such that x1 and x2 both are non-negative)

In [ ]:
# For this, we split each variable of A into 2
# We also need to add additional constraints that each of the new variable is >= 0 (or negative of that variable is <= 0)
# Which means we need a total of 2*n columns (one for each variable)
# We need m + 2*n rows, (m for the existing constraints, the rest 2*n to ensure each variable >= 0) for A
A_new = np.zeros((m + 2*n,2*n))

# We need m + 2*n size vector for B
B_new = np.zeros(m + 2*n)

for i in range(m):
  for j in range(n):
    # Making each variable of A into 2
    A_new[i,2*j] = A[i,j]
    A_new[i,2*j+1] = -A[i,j]

    # Copying the inital constraints into B
    B_new[i] = B[i]

# Handling constrains that each variable should be non-negative
for i in range(m, m + 2*n, 1):
  A_new[i,i-m] = -1
  B_new[i] = 0

# We also need to ensure that the cost function is updated
c_new = np.zeros(2*n)
for i in range(n):
  c_new[2*i] = c[i]
  c_new[2*i+1] = -c[i]


A = A_new.copy()
B = B_new.copy()
c = c_new.copy()

m = m + 2*n
n = 2*n

print("n =", n)
print("m =", m)
print("c =", c)
print("A =\n", A)
print("B =", B)

n = 4
m = 9
c = [ 3. -3.  7. -7.]
A =
 [[ 1. -1.  0. -0.]
 [ 0. -0.  1. -1.]
 [ 2. -2.  1. -1.]
 [ 1. -1.  2. -2.]
 [-1.  1. -1.  1.]
 [-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]
B = [ 4.  4.  8.  4. -1.  0.  0.  0.  0.]


Now we need to remove the assumption that someone given us initial fesable point

In [ ]:
# Make a new system
A_new = np.zeros((m+1, n+1))
B_new = np.zeros(m+1)
for i in range(m):
  for j in range(n):
    A_new[i,j] = A[i,j]
  B_new[i] = B[i]

for i in range(m):
  A_new[i, n] = 1

for j in range(n):
  A_new[m, j] = 0

A_new[m, n] = -1
B_new[m] = -min(B)

z_new = np.zeros(n+1)
z_new[n] = max(0, -min(B))

# New objective is to maximize z
c_new = np.zeros(n+1)
c_new[n] = 1

if z_new[n] == 0:
  z = np.zeros(n)
  print(f"Since all elements in B are positive, {z} works as initial feasable point")
else:
  vertex, objective = simplex(A_new, B_new, c_new, z_new, False)
  if objective < 0:
    print("No Solution")
  elif objective == np.inf:
    print("Modified problem is Unbounded, hence output is unbounded")
    z = [np.inf]
  else:
    z = vertex[:-1]

print(z)

Applying Simplex with epsilon = 0.1
Trying to reach a vertex before applying Simplex - Modified Problem. Moving to:  [1.50500355 1.50500445 4.01       1.50500455 1.50500455]
Vertex Visited: [1.5 1.5 4.  1.5 1.5], Objective Function = 1.5
Vertex Visited: [-6. -1.  4. -1. -1.], Objective Function = -1.0
Vertex Visited: [4.  1.5 1.5 1.5 1.5], Objective Function = 1.5
Vertex Visited: [4.  1.5 1.5 1.5 1.5], Objective Function = 1.5
[4.  1.5 1.5 1.5]


Now that we got the z, we have everything to solve the simplex algorithm

In [ ]:
if len(z) ==1 and z == [np.inf]:
  print("Modified problem is Unbounded, hence optimum is unbounded")
else:
  vertex, optimal_objective = simplex(A, B, c, z, True)

Applying Simplex with epsilon = 0.1
Trying to reach a vertex before applying Simplex. Moving to:  [ 4.1   -0.199]
Vertex Visited: [4. 0.], Objective Function = 12.0
Vertex Visited: [4. 0.], Objective Function = 12.0
Vertex Visited: [4. 0.], Objective Function = 12.0
Vertex Visited: [0. 2.], Objective Function = 14.0
Vertex Visited: [-2.  3.], Objective Function = 15.0


The sequence of vertices visited is printed above

In [ ]:
if len(z) == 1 and z == [np.inf]:
  print("Modified problem is Unbounded, hence optimum is unbounded")
else:
  print(f"Initial feasible point: {convert_vertex(z, n)}")
  if optimal_objective == np.inf:
    print("Optimum is unbounded")
  else:
    print(f"Optimal Vertex: {np.round(vertex, 9)}")
    print(f"Optimal Objective function: {np.round(optimal_objective, 9)}")

Initial feasible point: [2.5 0. ]
Optimal Vertex: [-2.  3.]
Optimal Objective function: 15.0
